# **Full Truck Dispatching**

In [3]:
import numpy as np
from scipy.optimize import linprog

def get_T():
    T = {}
    for j in range(J):
        for i in range(I):
            key = (j, i)
            TA_kji = []
            for k in range(K):
                TA = TD + (D[j][i] * 1000) / (S[k] * 1000 / 60)
                TA_kji.append((k, TA))
            TA_kji.sort(key=lambda x: x[1])
            T[key] = TA_kji
    return T

def get_w(T):
    w_kji = np.zeros((K, J, I))
    for key in T:
        j, i = key
        TA = T[key]
        w = []
        for n in range(len(TA)):
            if n == 0:
                w.append(0)
            else:
                W_n = max(TA[n-1][1] + w[n-1] + TL - TA[n][1], 0)
                w.append(W_n)
        for idx, (k, _) in enumerate(TA):
            w_kji[k, j, i] = w[idx]
    return w_kji

def get_o(T, T_now):
    o_ji = np.zeros((J, I))
    for j in range(J):
        for i in range(I):
            key = (j, i)
            TA = T[key]
            if not TA:
                L_ji = 0
            else:
                L_ji = TA[-1][1]
            C_jil = sum(C[k] for k, _ in TA)
            P_ji = P[j][i]
            o_ji[j][i] = T_now - (L_ji + C_jil / P_ji)
    return o_ji

def get_m():
    m_kji = np.zeros((K, J, I))
    for k in range(K):
        for j in range(J):
            for i in range(I):
                m_kji[k, j, i] = D[j, i] * M[k] / 100
    return m_kji


P1, P2, P3 = 0.5, 0.1, 0.4

K = 4
J = 3
I = 2

S = np.array([60, 50, 70, 65])
C = np.array([20, 25, 15, 30])
M = np.array([30, 25, 35, 28])

TD = 10
TL = 15
T_now = 120

D = np.array([
    [15, 20],
    [25, 10],
    [18, 30]
])

P = np.array([
    [100, 150],
    [200, 50],
    [75, 125]
])

T = get_T()
w_kji = get_w(T)
o_ji = get_o(T, T_now)
m_kji = get_m()

f1_min = np.min(w_kji)
f1_max = np.max(w_kji)
f2_min = np.min(o_ji)
f2_max = np.max(o_ji)
f3_min = np.min(m_kji)
f3_max = np.max(m_kji)

f1_opt = (w_kji - f1_min) / (f1_max - f1_min)
f2_opt = (np.repeat(o_ji[np.newaxis, :, :], K, axis=0) - f2_min) / (f2_max - f2_min)
f3_opt = (m_kji - f3_min) / (f3_max - f3_min)

y_min = P1 * f1_opt + P2 * f2_opt + P3 * f3_opt
y_min = y_min.flatten()

integrality = np.ones(K * I * J, dtype=int)
bounds = [(0, 1) for _ in range(K * I * J)]

A1 = []
b1 = []
for k in range(K):
    row = np.zeros(K * I * J)
    for j in range(J):
        for i in range(I):
            idx = k * (I * J) + j * I + i
            row[idx] = 1
    A1.append(row)
    b1.append(1)

A2 = []
b2 = []
for j in range(J):
    for i in range(I):
        row = np.zeros(K * I * J)
        for k in range(K):
            idx = k * (I * J) + j * I + i
            row[idx] = 1
        A2.append(row)
        b2.append(1)

row = np.ones(K * I * J)
A2.append(row)
b2.append(K)

opt = linprog(y_min,
              A_eq=np.array(A1),
              b_eq=np.array(b1),
              A_ub=np.array(A2),
              b_ub=np.array(b2),
              bounds=bounds,
              integrality=integrality,
              method='highs')

if opt.success:
    x_opt = opt.x.reshape((K, J, I))

    print("\nx_opt:\n")
    print("k\tj\ti\tx_ijk\n")
    for k in range(K):
        for j in range(J):
            for i in range(I):
                    print(f"{k+1}\t{j+1}\t{i+1}\t{int(x_opt[k,j,i])}")
        print("\n")
    print(f"y_min: {opt.fun:.2f}")
else:
    print("couldnt find x_opt")


x_opt:

k	j	i	x_ijk

1	1	1	1
1	1	2	0
1	2	1	0
1	2	2	0
1	3	1	0
1	3	2	0


2	1	1	0
2	1	2	1
2	2	1	0
2	2	2	0
2	3	1	0
2	3	2	0


3	1	1	0
3	1	2	0
3	2	1	0
3	2	2	1
3	3	1	0
3	3	2	0


4	1	1	0
4	1	2	0
4	2	1	0
4	2	2	0
4	3	1	1
4	3	2	0


y_min: 1.65
